In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#loading training data
all_genres_data = pd.read_csv('~/Downloads/COMP309A3-Data/allGenres.csv', index_col=0)

In [3]:
#importing testing data
unlabelled_data = pd.read_csv('~/Downloads/COMP309A3-Data/testing-instances.csv', index_col=0)

In [4]:
#replace the undefined values with NaN
all_genres_data.replace('?', np.nan, inplace=True)
all_genres_data["duration_ms"].replace(-1, np.nan, inplace=True)

unlabelled_data.replace('?', np.nan, inplace=True)
unlabelled_data["duration_ms"].replace(-1, np.nan, inplace=True)

In [5]:
#create new attribute
all_genres_data['Number in track_name'] = ''
unlabelled_data['Number in track_name'] = ''

In [6]:
#function to find if a string contain numbers
def has_digits(str):
    return any(char.isdigit() for char in str)

In [7]:
#fill the newly created attributed using the function from last cell
for i, name in all_genres_data['track_name'].iteritems():
    all_genres_data.at[i,'Number in track_name']=has_digits(name)
    
for i, name in unlabelled_data['track_name'].iteritems():
    unlabelled_data.at[i,'Number in track_name']=has_digits(name)

In [8]:
#create new attribute
all_genres_data['Japanese caharacter in artist_name'] = ''
unlabelled_data['Japanese caharacter in artist_name'] = ''

In [9]:
#function to find if a string contain japanese alphabets
import re
pattern = "[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]"
def has_jap_char(str):
    if(re.search(pattern, str)):
        return True
    else:
        return False

In [10]:
#fill the newly created attributed using the function from last cell
for i, name in all_genres_data['artist_name'].iteritems():
    all_genres_data.at[i,'Japanese caharacter in artist_name']=has_jap_char(name)
    
for i, name in unlabelled_data['artist_name'].iteritems():
    unlabelled_data.at[i,'Japanese caharacter in artist_name']=has_jap_char(name)

In [11]:
#remove unnecessary attributes
all_genres_data.drop(['track_hash', 'obtained_date', 'Genre', 'track_name','artist_name'], axis=1, inplace=True)
unlabelled_data.drop(['track_hash', 'obtained_date', 'track_name','artist_name'], axis=1, inplace=True)

In [12]:
#create the x and y and training and testing data from the data with samples
X=all_genres_data[all_genres_data.columns[:-3]]
X['Number in track_name'] = all_genres_data['Number in track_name']
X['Japanese caharacter in artist_name'] = all_genres_data['Japanese caharacter in artist_name']
y=all_genres_data[all_genres_data.columns[-3]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [13]:
#encode the categorial attributes using label encoder
from sklearn.preprocessing import LabelEncoder

preprocessed_X_train = X_train.copy()

encoder1 = LabelEncoder()
preprocessed_X_train['key'] = encoder1.fit_transform(preprocessed_X_train['key'].values.reshape(-1,1))

encoder2 = LabelEncoder()
preprocessed_X_train['mode'] = encoder2.fit_transform(preprocessed_X_train['mode'].values.reshape(-1,1))

encoder3 = LabelEncoder()
preprocessed_X_train['Number in track_name'] = encoder3.fit_transform(preprocessed_X_train['Number in track_name'].values.reshape(-1,1))

encoder4 = LabelEncoder()
preprocessed_X_train['Japanese caharacter in artist_name'] = encoder4.fit_transform(preprocessed_X_train['Japanese caharacter in artist_name'].values.reshape(-1,1))

In [14]:
#impute the missing values using KNN imputer
from sklearn.impute import KNNImputer

imputer = KNNImputer()
preprocessed_X_train = imputer.fit_transform(preprocessed_X_train)

preprocessed_X_train = pd.DataFrame(preprocessed_X_train, columns = X_train.columns)

In [15]:
#normalize the data using standard scaler
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

preprocessed_X_train = scaler.fit_transform(preprocessed_X_train)

preprocessed_X_train = pd.DataFrame(preprocessed_X_train, columns = X_train.columns)

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,Number in track_name,Japanese caharacter in artist_name
0,0.564371,-0.897725,0.046255,-0.110160,1.091925,-0.531099,1.624957,0.604162,0.634543,-0.749535,-0.501281,0.234378,-1.275796,-0.360730,-0.0382
1,0.821074,0.674327,0.848079,0.131605,0.320641,-0.558988,-1.546245,0.240191,0.521798,1.334160,-0.585294,0.496462,0.826217,-0.360730,-0.0382
2,0.756898,-0.350370,-0.503246,-0.307808,1.035213,-0.558988,1.336666,-0.754251,0.759815,1.334160,0.058146,-1.364182,0.271351,-0.360730,-0.0382
3,-0.013211,0.217605,0.836865,-0.826373,0.214779,-0.558988,1.336666,-0.555609,0.022500,-0.749535,-0.555642,0.332316,1.620041,-0.360730,-0.0382
4,0.371844,0.012666,-1.148069,0.390692,-0.027193,-0.558988,-1.546245,-0.774609,-0.090244,-0.749535,-0.470641,-1.276809,0.129596,2.772159,-0.0382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.371844,-0.657193,0.393899,0.091153,-0.061220,0.772424,1.336666,0.641176,0.177382,-0.749535,-0.641632,-0.276711,-0.490072,-0.360730,-0.0382
34996,1.847886,-0.878205,0.057470,-0.452306,0.135382,-0.558988,-0.969663,-0.549440,0.789913,-0.749535,1.093976,-0.145163,1.713194,-0.360730,-0.0382
34997,0.243492,-0.897742,-1.809714,-0.570449,1.435978,-0.540354,0.471792,1.264244,0.551245,1.334160,0.024541,1.476041,0.591310,-0.360730,-0.0382
34998,-0.654969,1.599482,1.408795,-1.206824,-0.204890,1.716404,-0.969663,-0.561778,-0.785585,1.334160,-0.324360,-0.344041,-0.943686,-0.360730,-0.0382


In [16]:
#do all 3 above steps on the testing data generated from the sample data
preprocessed_X_test = X_test.copy()
preprocessed_X_test['key'] = encoder1.transform(preprocessed_X_test['key'].values.reshape(-1,1))
preprocessed_X_test['mode'] = encoder2.transform(preprocessed_X_test['mode'].values.reshape(-1,1))
preprocessed_X_test['Number in track_name'] = encoder3.transform(preprocessed_X_test['Number in track_name'].values.reshape(-1,1))
preprocessed_X_test['Japanese caharacter in artist_name'] = encoder4.transform(preprocessed_X_test['Japanese caharacter in artist_name'].values.reshape(-1,1))
preprocessed_X_test = imputer.transform(preprocessed_X_test)
preprocessed_X_test = pd.DataFrame(preprocessed_X_test, columns = X_test.columns)
preprocessed_X_test = scaler.transform(preprocessed_X_test)
preprocessed_X_test = pd.DataFrame(preprocessed_X_test, columns = X_test.columns)

In [17]:
#do all 3 above steps on the unlabelled testing data 
prediction_X = unlabelled_data.copy()
prediction_X['key'] = encoder1.transform(prediction_X['key'].values.reshape(-1,1))
prediction_X['mode'] = encoder2.transform(prediction_X['mode'].values.reshape(-1,1))
prediction_X['Number in track_name'] = encoder3.transform(prediction_X['Number in track_name'].values.reshape(-1,1))
prediction_X['Japanese caharacter in artist_name'] = encoder4.transform(prediction_X['Japanese caharacter in artist_name'].values.reshape(-1,1))
prediction_X = imputer.transform(prediction_X)
prediction_X = pd.DataFrame(prediction_X, columns = unlabelled_data.columns)
prediction_X = scaler.transform(prediction_X)
prediction_X = pd.DataFrame(prediction_X, columns = unlabelled_data.columns)

In [ ]:
#train the data using different classification steps and print the score on the labelled testing data
from sklearn.metrics import accuracy_score

In [23]:
#Descision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=9) 
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.5235333333333333


In [24]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(max_depth=5) 
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.5767333333333333


In [25]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=11) 
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.57


In [79]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-10) 
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.5844666666666667


In [27]:
#logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=20) 
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.5371333333333334


In [28]:
#gaussian nb
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB(var_smoothing=0.1)
clf.fit(preprocessed_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(preprocessed_X_test), y_test))

Accuracy =  0.408


In [30]:
#applying pca on training, labelled testing, and unlabelled testing set
from sklearn.decomposition import PCA

pca = PCA(n_components=12) 

pca_X_train = preprocessed_X_train.copy()  
pca_X_train = pca.fit_transform(pca_X_train)
pca_X_train = pd.DataFrame(data = pca_X_train, columns = ['pc1', 'pc2','pc3', 'pc4','pc5', 'pc6','pc7', 'pc8','pc9', 'pc10','pc10', 'pc12'])

pca_X_test = pca.transform(preprocessed_X_test)
pca_X_test = pd.DataFrame(pca_X_test, columns = ['pc1', 'pc2','pc3', 'pc4','pc5', 'pc6','pc7', 'pc8','pc9', 'pc10','pc10', 'pc12'])
pca_prediction_X = pca.transform(prediction_X)
pca_prediction_X = pd.DataFrame(pca_prediction_X, columns = ['pc1', 'pc2','pc3', 'pc4','pc5', 'pc6','pc7', 'pc8','pc9', 'pc10','pc10', 'pc12'])

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc10,pc12
0,-1.196388,1.011413,-0.708417,1.190551,-0.707192,1.259696,0.395907,0.220195,-0.889885,-0.203701,-0.315645,-0.528016
1,-1.026187,-0.515279,0.905952,-1.410787,0.083234,-0.901715,-0.263673,0.078458,0.259970,-1.093910,1.116392,-0.096906
2,-1.099140,-0.560536,-0.090503,-0.354540,-0.369900,1.843396,-0.864509,0.004460,-0.799378,-0.689311,-0.827599,0.528619
3,-0.977958,-0.667145,-2.051049,0.458959,0.217149,0.012992,-0.642503,-0.832804,0.556027,-0.256149,0.424942,0.176050
4,1.222311,-0.112041,0.203597,0.068124,-0.289260,-1.448974,0.026956,1.646550,0.766665,-1.543066,-2.232970,0.535390
...,...,...,...,...,...,...,...,...,...,...,...,...
34995,-0.131225,0.416822,-0.799778,0.821790,-0.390009,0.853042,-0.480115,0.236524,-0.481117,0.369903,0.100377,-1.252491
34996,-2.126505,-1.266956,-0.077288,0.120613,-0.075629,-0.976903,0.487684,0.709154,0.570756,-0.089286,-0.215960,-0.281232
34997,-1.565749,1.964559,0.844420,0.073650,0.176937,0.398413,0.239678,-1.537540,-0.650645,-1.207351,0.136792,0.055829
34998,1.505370,-1.434952,0.481802,-1.814601,0.605336,-0.273072,-0.227568,-1.028233,-0.726946,0.839408,0.129895,-0.874593


In [ ]:
# Performing classification using the pca-based transformed data

In [32]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=9) 
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.44106666666666666


In [34]:
#Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(max_depth=5) 
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.5128


In [35]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=11) 
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.5079333333333333


In [36]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-6) 
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.5706


In [37]:
#logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=20) 
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.5294666666666666


In [38]:
#gaussian NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB(var_smoothing=0.1)
clf.fit(pca_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(pca_X_test), y_test))

Accuracy =  0.39193333333333336


In [39]:
#applying ica on training, labelled testing, and unlabelled testing set
from sklearn.decomposition import FastICA

ica = FastICA(n_components=12) 

ica_X_train = preprocessed_X_train.copy()  
ica_X_train = ica.fit_transform(ica_X_train)
ica_X_train = pd.DataFrame(data = ica_X_train, columns = ['ic1', 'ic2','ic3', 'ic4','ic5', 'ic6','ic7', 'ic8','ic9', 'ic10','ic10', 'ic12'])

ica_X_test = ica.transform(preprocessed_X_test)
ica_X_test = pd.DataFrame(ica_X_test, columns = ['ic1', 'ic2','ic3', 'ic4','ic5', 'ic6','ic7', 'ic8','ic9', 'ic10','ic10', 'ic12'])
ica_prediction_X = ica.transform(prediction_X)
ica_prediction_X = pd.DataFrame(ica_prediction_X, columns = ['ic1', 'ic2','ic3', 'ic4','ic5', 'ic6','ic7', 'ic8','ic9', 'ic10','ic10', 'ic12'])

,ic1,ic2,ic3,ic4,ic5,ic6,ic7,ic8,ic9,ic10,ic10,ic12
0,0.002482,-0.001973,-0.001654,0.003351,-0.002239,-0.008140,0.002012,0.008685,-0.000166,0.001689,-0.000888,-0.004186
1,0.001063,-0.004196,0.000697,0.003045,-0.001652,0.007553,0.004776,-0.005752,-0.000200,-0.003248,-0.002488,0.007528
2,0.004484,-0.001903,-0.002382,0.002563,0.003907,-0.007717,-0.000792,0.004389,-0.000350,-0.001431,0.006931,0.006900
3,0.000395,-0.002945,-0.003871,0.001175,0.002801,-0.007036,0.000418,-0.009111,-0.000252,-0.001867,-0.000870,-0.004106
4,0.013923,-0.003909,0.001962,-0.010643,0.004032,0.008527,-0.001746,0.003620,-0.000228,-0.004687,0.005731,-0.003516
...,...,...,...,...,...,...,...,...,...,...,...,...
34995,-0.001686,-0.002455,-0.000159,0.001388,-0.003210,-0.006624,0.004903,0.002764,-0.000183,0.006674,0.001677,-0.004275
34996,0.005300,0.003016,-0.002000,0.001683,0.002612,0.005627,0.005322,-0.002817,-0.000399,-0.000188,0.000357,-0.003938
34997,0.003712,-0.003665,-0.004440,0.001545,-0.006951,-0.003341,-0.004256,0.003767,-0.000371,-0.001264,-0.008702,0.007172
34998,-0.010146,0.000677,-0.005970,-0.002388,0.002504,0.004835,0.002470,-0.002549,-0.000098,0.006587,0.002209,0.007027


In [ ]:
# Performing classification using the pca-based transformed data


In [43]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=9) 
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.4852


In [44]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(max_depth=5) 
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.5518


In [45]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=11) 
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.5483333333333333


In [46]:
#MLP Classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-6) 
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.5345333333333333


In [47]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=20) 
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.48313333333333336


In [48]:
#Gaussian NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB(var_smoothing=0.1)
clf.fit(ica_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(ica_X_test), y_test))

Accuracy =  0.3828


In [49]:
!pip install gplearn

In [68]:
#applying gp on training, labelled testing, and unlabelled testing set
from gplearn.genetic import SymbolicTransformer

gp = SymbolicTransformer(n_components=5)

gp_X_train = preprocessed_X_train.copy()  
gp_X_train = gp.fit_transform(gp_X_train, LabelEncoder().fit_transform(y_train.values.reshape(-1,1)))
gp_X_train = pd.DataFrame(data = gp_X_train, columns = ['gp1', 'gp2','gp3', 'gp4','gp5'])

gp_X_test = gp.transform(preprocessed_X_test)
gp_X_test = pd.DataFrame(gp_X_test, columns = ['gp1', 'gp2','gp3', 'gp4','gp5'])
gp_prediction_X = gp.transform(prediction_X)
gp_prediction_X = pd.DataFrame(gp_prediction_X, columns = ['gp1', 'gp2','gp3', 'gp4','gp5'])

,gp1,gp2,gp3,gp4,gp5
0,0.564371,0.564371,0.564371,0.564371,0.564371
1,0.821074,0.821074,0.821074,0.821074,0.821074
2,0.756898,0.756898,0.756898,0.756898,0.756898
3,-0.013211,-0.013211,-0.013211,-0.013211,-0.013211
4,0.371844,0.371844,0.371844,0.371844,0.371844
...,...,...,...,...,...
34995,0.371844,0.371844,0.371844,0.371844,0.371844
34996,1.847886,1.847886,1.847886,1.847886,1.847886
34997,0.243492,0.243492,0.243492,0.243492,0.243492
34998,-0.654969,-0.654969,-0.654969,-0.654969,-0.654969


In [54]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=9) 
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.2894


In [55]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(max_depth=5) 
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.2782


In [56]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=11) 
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.2783333333333333


In [69]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-6) 
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.3024


In [58]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=20) 
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.2576


In [59]:
#Gaussian NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB(var_smoothing=0.1)
clf.fit(gp_X_train, y_train)
print("Accuracy = ", accuracy_score(clf.predict(gp_X_test), y_test))

Accuracy =  0.28326666666666667


In [ ]:
#Findin the scores of diffrenet classifier using variety of values for their key parameter on different sets of input

In [70]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [71]:
names = ["KNeighborsClassifier", "GaussianNB","Logistic Regression",
         "Decision Tree", "GradientBoosting", "Random Forest",
         "MLP"]

controllerNames = ["n_neighbors", "var_smoothing",
         "C", "max_depth", "max_depth",
         "max_depth", "alpha"]

controllers = [
    [1,2,3,4,5],
    [1e-9,1e-5,1e-1],
    [.1,.5,1.0,2.0, 5.0],
    [1,2,3,4,5,6,7,8,9,10],
    [1,2,3,4,5,6,7,8,9,10],
    [1,2,3,4,5,6,7,8,9,10],
    [1e-5, 1e-3,0.1, 10.0]
]

In [72]:
def classifierFinder(v, i):
    if v==0:
        return KNeighborsClassifier(n_neighbors=i)
    if v==1:
        return GaussianNB(var_smoothing=i)
    if v==2:
        return LogisticRegression(C=i) 
    if v==3:
        return DecisionTreeClassifier(max_depth=i+1) 
    if v==4:
        return GradientBoostingClassifier(max_depth=i+1) 
    if v==5:
        return RandomForestClassifier(max_depth=i+1) 
    if v==6:
        return MLPClassifier(alpha=i) 

In [74]:
predictions = []
for ctrl_cnt, ctrl in enumerate(controllers):
    cls_predictions = []
    for i in ctrl:
            clf = classifierFinder(ctrl_cnt, i)
            clf.fit(preprocessed_X_train, y_train)
            score=accuracy_score(clf.predict(preprocessed_X_test), y_test)
            cls_predictions.append(score)
    print(cls_predictions)
    

[0.41413333333333335, 0.41733333333333333, 0.44566666666666666, 0.46946666666666664, 0.4846]
[0.37046666666666667, 0.3734, 0.408]
[0.5374, 0.5368666666666667, 0.5371333333333334, 0.5371333333333334, 0.5370666666666667]
[0.3166, 0.3698666666666667, 0.4169333333333333, 0.4716666666666667, 0.49273333333333336, 0.5117333333333334, 0.5202666666666667, 0.5234, 0.5213333333333333, 0.5181333333333333]
[0.5696666666666667, 0.5811333333333333, 0.5811333333333333, 0.5754, 0.5749333333333333, 0.5647333333333333, 0.5566, 0.5554666666666667, 0.5542, 0.553]
[0.41313333333333335, 0.4688, 0.5014, 0.5252666666666667, 0.5382, 0.5564666666666667, 0.5612, 0.5653333333333334, 0.5714, 0.5712666666666667]
[0.5855333333333334, 0.5867333333333333, 0.5876666666666667, 0.5015333333333334]


In [75]:
predictions = []
for ctrl_cnt, ctrl in enumerate(controllers):
    cls_predictions = []
    for i in ctrl:
            clf = classifierFinder(ctrl_cnt, i)
            clf.fit(pca_X_train, y_train)
            score=accuracy_score(clf.predict(pca_X_test), y_test)
            cls_predictions.append(score)
    print(cls_predictions)
    

[0.40386666666666665, 0.4023333333333333, 0.43106666666666665, 0.4554, 0.4629333333333333]
[0.4178, 0.4178, 0.39193333333333336]
[0.5127333333333334, 0.5182666666666667, 0.5215333333333333, 0.5255333333333333, 0.5276666666666666]
[0.24633333333333332, 0.3132666666666667, 0.3542666666666667, 0.39026666666666665, 0.4129333333333333, 0.42606666666666665, 0.4312, 0.4401333333333333, 0.4446, 0.453]
[0.4962666666666667, 0.51, 0.5067333333333334, 0.5136666666666667, 0.5068, 0.5082666666666666, 0.5056666666666667, 0.4988, 0.5021333333333333, 0.4966]
[0.4032, 0.4182, 0.4358666666666667, 0.45526666666666665, 0.4686, 0.4788, 0.48833333333333334, 0.4975333333333333, 0.5039333333333333, 0.5074]
[0.5722, 0.5706666666666667, 0.5680666666666667, 0.4938666666666667]


In [76]:
predictions = []
for ctrl_cnt, ctrl in enumerate(controllers):
    cls_predictions = []
    for i in ctrl:
            clf = classifierFinder(ctrl_cnt, i)
            clf.fit(ica_X_train, y_train)
            score=accuracy_score(clf.predict(ica_X_test), y_test)
            cls_predictions.append(score)
    print(cls_predictions)
    

[0.4063333333333333, 0.40413333333333334, 0.4305333333333333, 0.4563333333333333, 0.46686666666666665]
[0.377, 0.37633333333333335, 0.3828]
[0.4441333333333333, 0.44606666666666667, 0.44953333333333334, 0.4548, 0.46653333333333336]
[0.231, 0.2975333333333333, 0.36046666666666666, 0.40726666666666667, 0.42806666666666665, 0.4486, 0.4738, 0.4852666666666667, 0.4962666666666667, 0.4991333333333333]
[0.5346, 0.5457333333333333, 0.5501333333333334, 0.5462, 0.5396, 0.5365333333333333, 0.5328666666666667, 0.5330666666666667, 0.5305333333333333, 0.527]
[0.3868666666666667, 0.42933333333333334, 0.4604666666666667, 0.4849333333333333, 0.5013333333333333, 0.5194, 0.5277333333333334, 0.5326666666666666, 0.5386666666666666, 0.5459333333333334]
[0.5326666666666666, 0.5315333333333333, 0.45726666666666665, 0.1]


In [ ]:
#Saving the prediciton to a csv file for submission
prediction_y = clf.predict(prediction_X)
prediction = pd.DataFrame(prediction_X.index+1,prediction_y)
display(prediction)
prediction.to_csv('answer.csv')